<a href="https://colab.research.google.com/github/chettkulkarni/deep_learning/blob/master/assignment%204/PyTorch_VGG16_Cifar_Assignment4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable
from PIL import Image
import torch.nn.functional as F
import numpy as np

In [0]:
class VGG16(nn.Module):
    def __init__(self):
        super(VGG16, self).__init__()
        self.conv1_1 = nn.Conv2d(3, 64, kernel_size=3, padding=1)
        self.conv1_2 = nn.Conv2d(64, 64, kernel_size=3, padding=1)
        self.conv2_1 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.conv2_2 = nn.Conv2d(128, 128, kernel_size=3, padding=1)
        self.conv3_1 = nn.Conv2d(128, 256, kernel_size=3, padding=1)
        self.conv3_2 = nn.Conv2d(256, 256, kernel_size=3, padding=1)
        self.conv3_3 = nn.Conv2d(256, 256, kernel_size=3, padding=1)
        self.conv4_1 = nn.Conv2d(256, 512, kernel_size=3, padding=1)
        self.conv4_2 = nn.Conv2d(512, 512, kernel_size=3, padding=1)
        self.conv4_3 = nn.Conv2d(512, 512, kernel_size=3, padding=1)

        self.pool = nn.MaxPool2d(2, 2)

        # fully conected layers:
        self.fc6 = nn.Linear(2048, 4096)
        self.fc7 = nn.Linear(4096, 4096)
        self.fc8 = nn.Linear(4096, 1000)

    def forward(self, x, training=True):
        x = F.relu(self.conv1_1(x))
        x = F.relu(self.conv1_2(x))
        x = self.pool(x)
        x = F.relu(self.conv2_1(x))
        x = F.relu(self.conv2_2(x))
        x = self.pool(x)
        x = F.relu(self.conv3_1(x))
        x = F.relu(self.conv3_2(x))
        x = F.relu(self.conv3_3(x))
        x = self.pool(x)
        x = F.relu(self.conv4_1(x))
        x = F.relu(self.conv4_2(x))
        x = F.relu(self.conv4_3(x))
        x = self.pool(x)
        x = x.view(x.size(0), -1) 
        x = F.relu(self.fc6(x))
        x = F.dropout(x, 0.5, training=training)
        x = F.relu(self.fc7(x))
        x = F.dropout(x, 0.5, training=training)
        x = self.fc8(x)
        return x
        return x

In [0]:
def load_data(train_batch_size, test_batch_size):
    # Fetch training data: total 60000 samples
    train_loader = torch.utils.data.DataLoader(
        datasets.CIFAR100('data', train=True, download=True,
                       transform=transforms.Compose([
                           transforms.Resize((32, 32)),
                           transforms.ToTensor(),
                           transforms.Normalize((0.1307,), (0.3081,))
                       ])),
        batch_size=train_batch_size, shuffle=True)

    # Fetch test data: total 10000 samples
    test_loader = torch.utils.data.DataLoader(
        datasets.CIFAR100('data', train=False, transform=transforms.Compose([
            transforms.Resize((32, 32)),
            transforms.ToTensor(),
            transforms.Normalize((0.1307,), (0.3081,))
        ])),
        batch_size=test_batch_size, shuffle=True)

    return (train_loader, test_loader)

In [0]:
def train(model, optimizer, epoch, train_loader, log_interval):
    model.train()

    # define loss function
    loss_fn = torch.nn.CrossEntropyLoss()

    # Iterate over batches of data
    for batch_idx, (data, target) in enumerate(train_loader):
        # Wrap the input and target output in the `Variable` wrapper
        data,target = images.to(device),labels.to(device)
        data, target = Variable(data), Variable(target)

        # Clear the gradients, since PyTorch accumulates them
        optimizer.zero_grad()

        # Forward propagation
        output = model(data)

        loss = loss_fn(output, target)

        # Backward propagation
        loss.backward()

        # Update the parameters(weight,bias)
        optimizer.step()

        # print log
        if batch_idx % log_interval == 0:
            print('Train set, Epoch {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                       100. * batch_idx / len(train_loader),
                loss.item()))

In [0]:
def test(model, epoch, test_loader):
    # State that you are testing the model; this prevents layers e.g. Dropout to take effect
    model.eval()

    # Init loss & correct prediction accumulators
    test_loss = 0
    correct = 0

    # define loss function
    loss_fn = torch.nn.CrossEntropyLoss(size_average=False)

    # Iterate over data
    for data, target in test_loader:
        data,target = images.to(device),labels.to(device)
        data, target = Variable(data), Variable(target)
        
        # Forward propagation
        output = model(data)

        # Calculate & accumulate loss
        test_loss += loss_fn(output, target).item()

        # Get the index of the max log-probability (the predicted output label)
        pred = np.argmax(output.cpu().data, axis=1)

        # If correct, increment correct prediction accumulator
        correct = correct + np.equal(pred, target.cpu().data).sum()

    # Print log
    test_loss /= len(test_loader.dataset)
    print('\nTest set, Epoch {} , Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(epoch,
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [0]:
torch.manual_seed(123)

model = VGG16()
model.cuda()
lr = 0.01
momentum=0.5
optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum)

train_batch_size = 64
test_batch_size = 1000
train_loader, test_loader = load_data(train_batch_size, test_batch_size)

Files already downloaded and verified


In [0]:
epochs = 10
log_interval = 100
for epoch in range(1, epochs + 1):
    train(model, optimizer, epoch, train_loader, log_interval=log_interval)
    test(model, epoch, test_loader)

Train set, Epoch 1 [0/50000 (0%)]	Loss: 0.078902
Train set, Epoch 1 [6400/50000 (13%)]	Loss: 0.098494
Train set, Epoch 1 [12800/50000 (26%)]	Loss: 0.094536
Train set, Epoch 1 [19200/50000 (38%)]	Loss: 0.101996
Train set, Epoch 1 [25600/50000 (51%)]	Loss: 0.090685
Train set, Epoch 1 [32000/50000 (64%)]	Loss: 0.092112
Train set, Epoch 1 [38400/50000 (77%)]	Loss: 0.087513
Train set, Epoch 1 [44800/50000 (90%)]	Loss: 0.091019


/usr/local/lib/python3.6/dist-packages/torch/nn/_reduction.py:43: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))



Test set, Epoch 1 , Average loss: 0.0108, Accuracy: 596/10000 (6%)

Train set, Epoch 2 [0/50000 (0%)]	Loss: 0.161239
Train set, Epoch 2 [6400/50000 (13%)]	Loss: 0.074130
Train set, Epoch 2 [12800/50000 (26%)]	Loss: 0.101728
Train set, Epoch 2 [19200/50000 (38%)]	Loss: 0.116841
Train set, Epoch 2 [25600/50000 (51%)]	Loss: 0.083146
Train set, Epoch 2 [32000/50000 (64%)]	Loss: 0.090985
Train set, Epoch 2 [38400/50000 (77%)]	Loss: 0.101729
Train set, Epoch 2 [44800/50000 (90%)]	Loss: 0.093101

Test set, Epoch 2 , Average loss: 0.0058, Accuracy: 610/10000 (6%)

Train set, Epoch 3 [0/50000 (0%)]	Loss: 0.092427
Train set, Epoch 3 [6400/50000 (13%)]	Loss: 0.054723
Train set, Epoch 3 [12800/50000 (26%)]	Loss: 0.032961
Train set, Epoch 3 [19200/50000 (38%)]	Loss: 3.400541
Train set, Epoch 3 [25600/50000 (51%)]	Loss: 0.034751
Train set, Epoch 3 [32000/50000 (64%)]	Loss: 0.013592
Train set, Epoch 3 [38400/50000 (77%)]	Loss: 0.001149
Train set, Epoch 3 [44800/50000 (90%)]	Loss: 0.000413

Test set,